# 对Honor_record实现聚类

### 环境准备

#### 常用环境导入

In [1]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score

In [2]:
from sklearn.decomposition import PCA

In [3]:
import pandas as pd

In [5]:
import numpy as np

In [6]:
import seaborn as sns

In [7]:
from sklearn.manifold import TSNE

##### 导入matplot

In [8]:
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

### 连接数据库

In [9]:
import sqlite3
conn = sqlite3.connect("../db.sqlite3")
cur = conn.cursor()
table_name = cur.execute("select name from sqlite_master where type='table' order by name").fetchall()
table_name

[('Training',),
 ('UserTemp',),
 ('account_confirmstring',),
 ('account_profile_awardhistory',),
 ('account_profile_awardhistory_User',),
 ('account_profile_competitionrecord',),
 ('account_profile_honortitle',),
 ('account_profile_paperrecord',),
 ('account_profile_patentrecord',),
 ('account_profile_publishrecord',),
 ('account_profile_researchproject',),
 ('account_user',),
 ('account_user_groups',),
 ('account_user_user_permissions',),
 ('auth_group',),
 ('auth_group_permissions',),
 ('auth_permission',),
 ('award_info_awardclassification',),
 ('award_info_awardinfo',),
 ('award_info_awardinfo_Major',),
 ('award_info_awardinfopost',),
 ('award_info_category',),
 ('award_info_post',),
 ('award_info_post_tags',),
 ('award_info_post_users_focus',),
 ('award_info_tag',),
 ('captcha_captchastore',),
 ('django_admin_log',),
 ('django_content_type',),
 ('django_migrations',),
 ('django_session',),
 ('grade_info_deptinfo',),
 ('grade_info_gradeinfo',),
 ('grade_info_lessoninfo',),
 ('grade

In [10]:
cur.execute("PRAGMA table_info(account_profile_honortitle)")
cur.fetchall()

[(0, 'id', 'integer', 1, None, 1),
 (1, 'honorName', 'varchar(25)', 1, None, 0),
 (2, 'proclaimOrganization', 'varchar(25)', 1, None, 0),
 (3, 'honorRank', 'varchar(2)', 1, None, 0),
 (4, 'issueDate', 'date', 1, None, 0),
 (5, 'type', 'varchar(2)', 1, None, 0),
 (6, 'SchoolYear_id', 'integer', 1, None, 0),
 (7, 'User_id', 'integer', 1, None, 0)]

### 构建CSV

##### 荣誉称号

In [59]:
csv_head = ["id", "User_id", "honorRank"]
record = conn.execute(("select {} from account_profile_honortitle where type == 'ho'".format(",".join(csv_head)))).fetchall()

##### 其他成果

In [87]:
csv_head = ["id", "User_id", "honorRank"]
record = conn.execute("select {} from account_profile_honortitle where type == 'ot'".format(",".join(csv_head))).fetchall()

In [34]:
import csv

In [35]:
def save_csv(record, csv_head, csv_name):
    with open(csv_name, "w", newline="") as f:
        ff = csv.writer(f)
        ff.writerow(csv_head)
        ff.writerows(record)

In [70]:
def read_record(csv_name, csv_head):
    df_record = pd.read_csv(csv_name, header=0, encoding="utf-8")
    mapping = list(map(lambda x:False if x <= 1 else True, [i for i in range(0, len(csv_head))]))
#     df_one_hot = pd.factorize(df_record.loc[:, mapping].copy())
    df_fact = df_record.loc[:, mapping].copy()
    df_one_hot = pd.get_dummies(df_record.loc[:, mapping].copy())
    return df_record, df_one_hot, df_fact

##### honor_title

In [60]:
save_csv(record, csv_head, "honor_record_honor_title.csv")

##### other_achievements

In [88]:
save_csv(record, csv_head, "honor_record_other_achievements.csv")

### read data_frame

##### honor_title

In [71]:
df_record, df_record_one_hot, df_record_fact = read_record("honor_record_honor_title.csv", csv_head)

##### other_achievements

In [89]:
df_record, df_record_one_hot, df_record_fact = read_record("honor_record_other_achievements.csv", csv_head)

show columns

In [90]:
df_record_fact.columns

Index(['honorRank'], dtype='object')

### Factorize 或Mapping 给 record 赋值

##### Mapping

In [91]:
honor_rank_mapping = {
    "in": 1,
    "na": 2,
    "ds": 3,
    "co": 4,
    "ac": 5,
    "ot": 6
}

df_record_fact["honorRank"] = df_record_fact["honorRank"].map(honor_rank_mapping)

##### factorize

In [ ]:
for i in df_record_fact.columns:
    df_record_fact[i] = pd.Series(pd.factorize(df_record_fact[i])[0])

### 保存 CSV

##### one-hot

In [80]:
df_record_one_hot['user_id'] = df_record.loc[:, ["User_id"]]
df_record_one_hot = df_record_one_hot.sort_values("user_id").reset_index(drop=True)
df_record_one_hot.to_csv("honor_title_one_hot.csv")

In [92]:
df_record_one_hot['user_id'] = df_record.loc[:, ["User_id"]]
df_record_one_hot = df_record_one_hot.sort_values("user_id").reset_index(drop=True)
df_record_one_hot.to_csv("other_achievements_one_hot.csv")

##### factorize

In [81]:
df_record_fact["user_id"] = df_record.loc[:, ['User_id']]
df_record_fact = df_record_fact.sort_values("user_id").reset_index(drop=True)
df_record_fact.to_csv("honor_title_fact.csv")

In [93]:
df_record_fact["user_id"] = df_record.loc[:, ['User_id']]
df_record_fact = df_record_fact.sort_values("user_id").reset_index(drop=True)
df_record_fact.to_csv("other_achievements_fact.csv")